In [16]:
import numpy as np
import sys
import random
from scipy.linalg import LinAlgError

## Cholesky Decomposition

In [17]:
def read_input(name = 'input.txt'): 
    with open(name, "r") as f:
        n = int(f.readline().strip())
        a = np.zeros((n, n), dtype=float)
        b = np.zeros(n, dtype=float)
        for i in range(n):
            a[i] = np.fromstring(f.readline(), sep=" ")
        b = np.fromstring(f.readline(), sep=" ")
    
    return a, b

In [18]:
def write_output(x, true_x, name = "output.txt"):
    abs_err = np.sqrt(np.sum((x - true_x) ** 2))
    rel_err = abs_err / np.sqrt(np.sum(true_x ** 2))

    with open(name, "w") as f:
        f.write("My Result:\t" + np.array2string(x, precision=4)+ "\n")
        f.write("NumPy Result: " + np.array2string(true_x, precision=4) + "\n")
        f.write(f"Absolute error: {abs_err:.6e}\n")
        f.write(f"Relative error: {rel_err:.6e}\n")
    

In [19]:
def decompose(a):
    h = np.zeros(np.shape(a), dtype=complex)
    n = len(h[0])
    
    for k in range(n):
        for i in range(k + 1):
            s = np.dot(h[i][0:i], h[k][0:i])
            
            if i == k:
                val_to_sqrt = a[i][i] - s
                h[i][i] = np.sqrt(val_to_sqrt)
            else:
                if h[i][i] == 0:
                    raise LinAlgError("Division by zero encountered. Matrix may be singular.")
                h[k][i] = (a[k][i] - s) / h[i][i]
    return h

In [20]:
def Choleski(A, B):
    a = A.astype(complex).copy()
    b = B.astype(complex).copy()
    n = len(a[0])
    
    if (a == a.T).all():
        h = decompose(a)
    else:
        raise LinAlgError("Matrix is not symmetric.")

    #Forward substitution 
    y = np.zeros(n, dtype=complex)
    for i in range(n):
        y[i] = (b[i] - np.dot(h[i, :i], y[:i])) / h[i, i]
    
    #Backward substitution
    x = np.zeros(n, dtype=complex)
    ht = h.T
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(ht[i, i+1:], x[i+1:])) / ht[i, i]
        
    return x

In [6]:
a, b = read_input()
write_output(Choleski(a, b), np.linalg.solve(a, b))

##### Тестування

In [7]:
def create(n: int) -> np.ndarray:
    if n <= 0:
        raise ValueError("Matrix dimension n must be a positive integer.") 
    m = np.random.rand(n, n)
    a = m @ m.T 
    b = np.random.rand(n)
    return a, b

In [8]:
a, b = create(10)
write_output(Choleski(a, b), np.linalg.solve(a, b), "tested.txt")

## Thomas

In [10]:
def read_and_parse(filepath):
#######################READ#######################
    try:
        with open(filepath, 'r') as f:
            lines = [line.strip() for line in f if line.strip()]

        # Parse n
        n = int(lines[0])

        # Check for consistent line numbers
        if len(lines) != n + 2:
            raise ValueError(f"File format error: Expected {n+2} data lines, but found {len(lines)}.")

        # Parse matrix A
        A_matrix = []
        for i in range(1, n + 1):
            row = [float(x) for x in lines[i].split()]
            if len(row) != n:
                raise ValueError(f"Matrix row {i-1} has incorrect number of elements.")
            A_matrix.append(row)

        # Parse vector B
        B_vector = [float(x) for x in lines[n + 1].split()]
        if len(B_vector) != n:
            raise ValueError("Vector B has incorrect number of elements.")

    except FileNotFoundError:
        raise FileNotFoundError(f"Error: File not found at {filepath}")
    except (ValueError, IndexError) as e:
        raise ValueError(f"Error parsing file: {e}")
        
#######################PARSE#######################
    for i in range(n):
        for j in range(n):
            if abs(i - j) > 1 and A_matrix[i][j] != 0:
                raise ValueError(f"Matrix is not tridiagonal. Element at ({i},{j}) is non-zero.")
    if n < 2: raise ValueError("The system must be at least 2x2.")

    d1 = A_matrix[0][0]
    if d1 == 0:
        raise ValueError("A[0][0] is zero, cannot derive first boundary condition.")
    c1 = A_matrix[0][1]
    b1 = B_vector[0]
    chi1 = -c1 / d1
    mu1 = b1 / d1

    dn = A_matrix[n-1][n-1]
    if dn == 0:
        raise ValueError(f"A[{n-1}][{n-1}] is zero, cannot derive second boundary condition.")
    an = A_matrix[n-1][n-2]
    bn = B_vector[n-1]
    chi2 = -an / dn
    mu2 = bn / dn

    A_coeff, C_coeff, B_coeff, f_coeff = [], [], [], []

    for i in range(1, n - 1):
        A_coeff.append(A_matrix[i][i-1])
        C_coeff.append(-A_matrix[i][i])
        B_coeff.append(A_matrix[i][i+1])
        f_coeff.append(-B_vector[i])

    return (A_coeff, C_coeff, B_coeff, f_coeff, chi1, mu1, chi2, mu2, d1, dn)

In [11]:
def solve_tridiagonal(A, C, B, f, chi1, mu1, chi2, mu2, d1, dn):
    n = len(C) + 1

    alpha = [0.0] * (n + 1)
    beta = [0.0] * (n + 1)

    alpha[1] = chi1
    beta[1] = mu1
    
    det_product = 1.0

    for i in range(1, n):
        denominator = C[i-1] - A[i-1] * alpha[i] #it is the same so we calculate it here
        if denominator == 0:
            raise ValueError("Division by zero in forward sweep. Method is not applicable.")
        
        # Formula (2.40)
        det_product *= denominator
        alpha[i+1] = B[i-1] / denominator
        beta[i+1] = (f[i-1] + A[i-1] * beta[i]) / denominator

    last_pivot = (1 - chi2 * alpha[n])
    determinant = (d1  * det_product * last_pivot) * dn * ((-1)**len(C))

    y = [0.0] * (n + 1)
    
    denominator_y_n = 1 - chi2 * alpha[n]
    if denominator_y_n == 0:
        raise ValueError("Division by zero in backward sweep. Unique solution may not exist.")
    
    y[n] = (mu2 + chi2 * beta[n]) / denominator_y_n
    for i in range(n - 1, -1, -1):
        y[i] = alpha[i+1] * y[i+1] + beta[i+1]
        
    y = np.array(y)
    return y, determinant

In [12]:
def write_output_tomas(x, true_x, name = "output.txt"):
    abs_err = np.sqrt(np.sum((x[0] - true_x) ** 2))
    rel_err = abs_err / np.sqrt(np.sum(true_x ** 2))

    with open(name, "w") as f:
        f.write("My Result:\t" + np.array2string(x[0], precision=9, separator=', ') + f"\nDeterminant: {x[1]}\n")
        f.write("NumPy Result: " + np.array2string(true_x, precision=9, separator=', ') + "\n")
        f.write(f"Absolute error: {abs_err:.6e}\n")
        f.write(f"Relative error: {rel_err:.6e}\n")

##### Тестування

In [14]:
def generate(n, filepath='generated_for_thomas.txt'):
    if n < 3: raise ValueError("System size 'n' must be at least 3 for a meaningful tridiagonal system.")
    A_matrix = [[0.0] * n for _ in range(n)]
    B_vector = [0.0] * n
    
    for i in range(n):
        lower = 0.0
        upper = 0.0
        if i > 0:
            A_matrix[i][i-1] = random.uniform(-10, 10)
            lower = abs(A_matrix[i][i-1])
        if i < n - 1:
            A_matrix[i][i+1] = random.uniform(-10, 10)
            upper = abs(A_matrix[i][i+1])
            
        diag_val = lower + upper + random.uniform(1, 10)
        A_matrix[i][i] = diag_val * random.choice([-1, 1]) # Give it a random sign
        B_vector[i] = random.uniform(-50, 50)

    try:
        with open(filepath, 'w') as f:
            f.write(f"{n}\n")
            for row in A_matrix:
                f.write(" ".join(f"{x:.4f}" for x in row) + "\n")
            f.write(" ".join(f"{x:.4f}" for x in B_vector) + "\n")
    except IOError as e:
        raise IOError(f"Error writing to file {filepath}: {e}")

    return filepath

In [15]:
g = generate(10)
write_output_tomas(solve_tridiagonal(*read_and_parse(g)), np.linalg.solve(*read_input(g)), "thomas.txt")